In [1]:
import glob
import json
import pandas as pd
from gensim.models import Word2Vec

### Functions

In [2]:
def evalList(dictionary):
    terms = []

    focus_term = list(dictionary.keys())[0]
    dictionary = dictionary[focus_term]

    for term in dictionary.keys():
        terms.append(term)

    return terms

### List

In [5]:
path_evalIntrinsic = "/home/hiwi/Dokumente/masterthesis/data/evaluation/eval_approach_1_key.json"
path_expert = '/home/hiwi/Dokumente/masterthesis/data/evaluation/expert/'

# Load vocabulary list
intrinsic_eval = json.loads(open(path_evalIntrinsic).read())
intrinsic_eval = intrinsic_eval["eval_list"]

# Expert voting
expert = {}

for key in intrinsic_eval.keys():
    filename = intrinsic_eval[key]
    terms = json.loads(open(path_expert+filename).read())
    expert[key] = evalList(terms)

In [12]:
path_evalExtrinsic = '/home/hiwi/Dokumente/masterthesis/data/terminology/eval_extrinsic.json'
# Load vocabulary list
extrinsic_eval = json.loads(open(path_evalExtrinsic).read())
extrinsic_eval = extrinsic_eval["eval_list"]


In [6]:
path_trained = '/home/hiwi/Dokumente/masterthesis/data/models/phase5/'
#path_trained = '/home/hiwi/Dokumente/masterthesis/archieve/ma_data/models/'
fileNames = glob.glob(path_trained+"03sentence*.bin")

In [7]:
len(fileNames)

26

In [8]:
models = []
for i in range(1, len(fileNames)+1):
    name = "model_" + str(i)
    models.append(name)
len(models)

26

In [9]:
for i in range(0, len(fileNames)):
    models[i] = Word2Vec.load(fileNames[i])

In [10]:
evalPairs = []

for key in expert.keys():
    for term in expert[key]:
        try:
            for model in models:
                score = model.wv.similarity(key, term)
            if [key, term] not in evalPairs:
                evalPairs.append([key, term])
        except:
            pass    

print(len(evalPairs))

751


In [28]:
evalPairs = []

for term in extrinsic_eval:
    cond = []
    for i in range(0,len(models)):
        model = models[i]
        test = (term in model.wv.vocab)
        cond.append(term in model.wv.vocab)
    if (cond[0] & cond[1] & cond[2]):
        evalPairs.append(term)
    
print(len(evalPairs))

48


In [11]:
pathResults = "/home/hiwi/Dokumente/masterthesis/data/evaluation/eval_approach_1_phase5.csv"

f = open(pathResults, "a")
for pair in evalPairs:
    newLine = ";".join(pair)
    f.write(newLine)
    f.write("\n")

f.close()

### List of unique terms

In [35]:
uniqueTerms = []

for pair in evalPairs:
    if pair[0] not in uniqueTerms:
        uniqueTerms.append(pair[0])
    if pair[1] not in uniqueTerms:
        uniqueTerms.append(pair[1])
len(uniqueTerms)

410

In [37]:
pathTerms = "/home/hiwi/Dokumente/masterthesis/data/terminology/eval_intrinsic_part3_2uniqueTerms.csv"

f = open(pathTerms, "a")
for term in uniqueTerms:
    f.write(term)
    f.write("\n")

f.close()

### Test file

In [14]:
test = pd.read_csv(pathResults, sep=";", names=["key", "term"])

In [18]:
pairs = []
for index, row in test.iterrows():
    pair = [row["key"], row["term"]]
    pairs.append(pair)